# 0. Introduction

In this project, we peform the following steps to explore data about the neighborhoods of Toronto, Ontario, Canada:
- scrape and parse information about the neighborhoods from Wikipedia using Beautifulsoup package
- obtain latitude and longitude information and combine with neighborhood info
- visualize the neighborhoods on a map using the Folium package
- use the Foursquare API to obtain detailed information about venues in each neighborhood
- explore and cluster the neighborhoods using the K-means clustering method.

## 1.  Scrape the webpage and pre-process the dataset


We start by installing the relevant packages:

In [1]:
!pip install bs4

import pandas as pd
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

We obtain the relevant webpage from Wikipedia:

In [2]:
html_data = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

We parse the html data using BeautifulSoup:

In [3]:
soup = BeautifulSoup(html_data, "html.parser")

Extract data from the table into three relevant columns:

In [4]:
table=soup.find('table')
table_contents=[]
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
# print(table_contents)

In [5]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A','East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business','EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto','MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


Remove rows where the borough is 'Not assigned':

In [6]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace=True)
#  reset index
df = df.reset_index(drop=True)
df.shape

(103, 3)


## 2.  Obtain the coordinates of the neighborhoods

Read in the coordinate data for Toronto postal codes from csv file:

In [7]:
# Obtain coordinates for Toronto neighborhoods 
#!pip install geopy
#from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
# postal_data = pd.read_csv('http://cocl.us/Geospatial_data') 
#- revisit geocoders later

# import from csv
toronto_coords = pd.read_csv("C:\\Users\\steph\\pythonstuff\\Geospatial_Coordinates.csv")
toronto_coords

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Merge the coordinate data with the neighborhood data.

In [8]:
df = pd.merge(df, toronto_coords, on='Postal Code')
df.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [9]:
df.shape    

(103, 5)

The main data set has 103 rows and 5 columns.

## 3.  Map of Toronto

First, let's download the dependencies that we will need.

In [10]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!pip install folium
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# import k-means from clustering stage
from sklearn.cluster import KMeans

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [11]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="course_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto, Ontario, Canada are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Toronto, Ontario, Canada are 43.6534817, -79.3839347.


In [12]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## 4.  Using the Foursquare API

We start with the required credentials:

In [13]:
CLIENT_ID = '3VH3RBUX11UHLC4BYM2THX4L52OB2U2D2QX1YKGMXQ0ZIPUR' # your Foursquare ID
CLIENT_SECRET = 'GPU4351JMFH2ZRBFW4S04CN3TYB2HLINMZDHX0T11IEWXID1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3VH3RBUX11UHLC4BYM2THX4L52OB2U2D2QX1YKGMXQ0ZIPUR
CLIENT_SECRET:GPU4351JMFH2ZRBFW4S04CN3TYB2HLINMZDHX0T11IEWXID1


The get request using the Foursquare API looks like the following:

https://api.foursquare.com/v2/venues/explore?client_id=CLIENT_ID&client_secret=CLIENT_SECRET&ll=LATITUDE,LONGITUDE&v=VERSION&limit=LIMIT

###  Exploring a sample neighborhood:

In [14]:
df.loc[2, 'Neighborhood']

'Regent Park, Harbourfront'

In [15]:
neighborhood_latitude = df.loc[2, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[2, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[2, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


Let's get the top venues that are in Regent Park, Harbourfront neighborhood within a radius of 500 meters.

In [16]:
# First, let's create the GET request URL. Name your URL url.
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)


In [17]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60c67bd34ad79a70cc290a28'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 42,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '53b8466a498e83df908c3f21',
       'name': 'Tandem Coffee',
       'location': {'address': '368 King St E',
        'crossStreet': 'at Trinity St',
        'lat': 43.65355870959944,
        'lng': -79.36180945913513,
        'labeledLatLngs': [{'label': 'display',
 

With the Foursquare API, all the information is in the items key. Let's use get_category_type function previously defined.

In [18]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Let's clean the json and structure it into a pandas dataframe.  We can see below the top venues in Regent Park, Harbourfront.

In [19]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

<ipython-input-19-53c7dde730be>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Tandem Coffee,Coffee Shop,43.653559,-79.361809
1,Roselle Desserts,Bakery,43.653447,-79.362017
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980
5,The Distillery Historic District,Historic Site,43.650244,-79.359323
6,Distillery Sunday Market,Farmers Market,43.650075,-79.361832
7,SOMA chocolatemaker,Chocolate Shop,43.650622,-79.358127
8,Sumach Espresso,Coffee Shop,43.658135,-79.359515
9,Arvo,Coffee Shop,43.649963,-79.361442


In [20]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

42 venues were returned by Foursquare.


###  Using the Foursquare API to obtain information on all neighborhoods.

Let's create a function to repeat the same process to all the neighborhoods in Toronto.

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [22]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

Let's check how many venues were returned for each neighborhood

In [23]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",16,16,16,16,16,16
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
...,...,...,...,...,...,...
Willowdale West,5,5,5,5,5,5
"Willowdale, Newtonbrook",1,1,1,1,1,1
Woburn,5,5,5,5,5,5


Let's find out how many unique categories can be curated from all the returned venues

In [24]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 263 uniques categories.


## 5.  Analyze the venues in each neighborhood:

In [25]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.


In [26]:
toronto_onehot.shape  

(1992, 263)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [27]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
# toronto_grouped        takes a while to render

In [28]:
toronto_grouped.shape    

(100, 263)

Let's print each neighborhood along with the top 5 most common venues

In [29]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge   0.2
1               Skating Rink   0.2
2             Breakfast Spot   0.2
3             Clothing Store   0.2
4  Latin American Restaurant   0.2


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.29
1  Sandwich Place  0.14
2    Dance Studio  0.14
3             Gym  0.14
4     Coffee Shop  0.14


----Bathurst Manor, Wilson Heights, Downsview North----
         venue  freq
0  Coffee Shop  0.12
1         Bank  0.12
2  Pizza Place  0.06
3  Gas Station  0.06
4     Pharmacy  0.06


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Bank  0.25
2   Chinese Restaurant  0.25
3                 Café  0.25
4    Mobile Phone Shop  0.00


----Bedford Park, Lawrence Manor East----
              venue  freq
0        Restaurant  0.09
1    Sandwich Place  0.09
2       Coffee Shop  0.09
3  Sushi Restaurant  0.04
4               Pub  0.04


----Bercz

          venue  freq
0         Field   0.2
1         Trail   0.2
2  Hockey Arena   0.2
3       Dog Run   0.2
4  Tennis Court   0.2


----India Bazaar, The Beaches West----
                  venue  freq
0  Fast Food Restaurant  0.11
1           Pizza Place  0.06
2    Italian Restaurant  0.06
3             Pet Store  0.06
4            Playground  0.06


----Kennedy Park, Ionview, East Birchmount Park----
               venue  freq
0     Discount Store  0.29
1        Bus Station  0.14
2         Hobby Shop  0.14
3        Coffee Shop  0.14
4  Convenience Store  0.14


----Kensington Market, Chinatown, Grange Park----
                           venue  freq
0                           Café  0.09
1                    Coffee Shop  0.05
2  Vegetarian / Vegan Restaurant  0.05
3                   Burger Joint  0.04
4                    Gaming Cafe  0.04


----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens----
                        venue  freq
0                        Pa

4  Vietnamese Restaurant  0.11


----The Annex, North Midtown, Yorkville----
            venue  freq
0  Sandwich Place  0.19
1     Coffee Shop  0.12
2            Café  0.12
3     Pizza Place  0.06
4        Pharmacy  0.06


----The Beaches----
                             venue  freq
0                Health Food Store  0.33
1                              Pub  0.33
2                      Yoga Studio  0.00
3               Mexican Restaurant  0.00
4  Molecular Gastronomy Restaurant  0.00


----The Danforth  East----
                 venue  freq
0                 Park  0.50
1         Intersection  0.25
2    Convenience Store  0.25
3  Martial Arts School  0.00
4       Massage Studio  0.00


----The Danforth West, Riverdale----
                venue  freq
0    Greek Restaurant  0.24
1  Italian Restaurant  0.07
2         Coffee Shop  0.07
3      Ice Cream Shop  0.05
4         Yoga Studio  0.02


----The Kingsway, Montgomery Road, Old Mill North----
                        venue  freq
0        

Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order.


In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Breakfast Spot,Skating Rink,Clothing Store,Latin American Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Sandwich Place,Dance Studio,Pub,Gym,Airport Food Court,Deli / Bodega,Escape Room,Electronics Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Park,Pizza Place,Shopping Mall,Bridal Shop,Sandwich Place,Diner,Sushi Restaurant,Mobile Phone Shop
3,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Women's Store,Department Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
4,"Bedford Park, Lawrence Manor East",Restaurant,Coffee Shop,Sandwich Place,Cupcake Shop,Indian Restaurant,Butcher,Café,Italian Restaurant,Sushi Restaurant,Liquor Store


## 6. Clustering Analysis of Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.


In [32]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 4, 2, 2, 2, 2, 2, 2, 2, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [33]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto data [df] to add lat/long for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# remove NAs and cast to type int
toronto_merged['Cluster Labels']=toronto_merged['Cluster Labels'].fillna(0).astype('int') 

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4,Park,Fast Food Restaurant,Food & Drink Shop,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
1,M4A,North York,Victoria Village,43.725882,-79.315572,2,Hockey Arena,Coffee Shop,Portuguese Restaurant,Intersection,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Park,Bakery,Café,Pub,Greek Restaurant,Sandwich Place,Bank,Spa,Beer Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,2,Clothing Store,Furniture / Home Store,Accessories Store,Miscellaneous Shop,Vietnamese Restaurant,Coffee Shop,Boutique,Distribution Center,Dessert Shop,Dim Sum Restaurant
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,2,Coffee Shop,Sushi Restaurant,Burrito Place,Yoga Studio,Theater,Sandwich Place,Salad Place,Diner,Burger Joint,Japanese Restaurant


Finally, let's visualize the resulting clusters

In [34]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examining the Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.


Cluster 0: 

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Etobicoke,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,York,0,Park,Women's Store,Pool,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Deli / Bodega
35,East York/East Toronto,0,Park,Intersection,Convenience Store,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Deli / Bodega
40,North York,0,Park,Snack Place,Airport,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Donut Shop
45,North York,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,North York,0,Park,Event Space,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
61,Central Toronto,0,Park,Bus Line,Swim School,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Ethiopian Restaurant
64,York,0,Park,Jewelry Store,Convenience Store,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Deli / Bodega
66,North York,0,Park,Moving Target,Convenience Store,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Deli / Bodega
68,Central Toronto,0,Park,Trail,Sushi Restaurant,Jewelry Store,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Dance Studio


Cluster 1: 

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Scarborough,1,Playground,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant


Cluster 2: 

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,2,Hockey Arena,Coffee Shop,Portuguese Restaurant,Intersection,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
2,Downtown Toronto,2,Coffee Shop,Park,Bakery,Café,Pub,Greek Restaurant,Sandwich Place,Bank,Spa,Beer Store
3,North York,2,Clothing Store,Furniture / Home Store,Accessories Store,Miscellaneous Shop,Vietnamese Restaurant,Coffee Shop,Boutique,Distribution Center,Dessert Shop,Dim Sum Restaurant
4,Queen's Park,2,Coffee Shop,Sushi Restaurant,Burrito Place,Yoga Studio,Theater,Sandwich Place,Salad Place,Diner,Burger Joint,Japanese Restaurant
7,North York,2,Japanese Restaurant,Gym,Café,Caribbean Restaurant,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Women's Store
...,...,...,...,...,...,...,...,...,...,...,...,...
96,Downtown Toronto,2,Pizza Place,Coffee Shop,Pharmacy,Chinese Restaurant,Bakery,Restaurant,Café,Pub,Italian Restaurant,Bank
97,Downtown Toronto,2,Coffee Shop,Sandwich Place,Café,Hotel,Bank,Japanese Restaurant,Asian Restaurant,Gym,Restaurant,Sushi Restaurant
99,Downtown Toronto,2,Sushi Restaurant,Japanese Restaurant,Coffee Shop,Restaurant,Indian Restaurant,Men's Store,Mediterranean Restaurant,Burrito Place,Gay Bar,Fast Food Restaurant
100,East Toronto Business,2,Light Rail Station,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Pizza Place,Restaurant


Cluster 3: Baseball Fields

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,North York,3,Baseball Field,Food Truck,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Women's Store,Falafel Restaurant
57,North York,3,Baseball Field,Furniture / Home Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Falafel Restaurant,Doner Restaurant
101,Etobicoke,3,Baseball Field,Women's Store,Department Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant


Cluster 4:

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,4,Park,Fast Food Restaurant,Food & Drink Shop,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
6,Scarborough,4,Fast Food Restaurant,Women's Store,Department Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant
27,North York,4,Pool,Mediterranean Restaurant,Fast Food Restaurant,Golf Course,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
47,East Toronto,4,Fast Food Restaurant,Board Shop,Pizza Place,Brewery,Sandwich Place,Burrito Place,Steakhouse,Restaurant,Italian Restaurant,Ice Cream Shop
63,York,4,Brewery,Bus Line,Pizza Place,Convenience Store,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
72,North York,4,Coffee Shop,Grocery Store,Pharmacy,Pizza Place,Supermarket,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
82,Scarborough,4,Pizza Place,Chinese Restaurant,Gas Station,Coffee Shop,Thai Restaurant,Fast Food Restaurant,Bank,Italian Restaurant,Dessert Shop,Dim Sum Restaurant
88,Etobicoke,4,Liquor Store,Pizza Place,Seafood Restaurant,Fast Food Restaurant,Mexican Restaurant,Flower Shop,Restaurant,American Restaurant,Café,Pharmacy
89,Etobicoke,4,Grocery Store,Beer Store,Coffee Shop,Video Store,Sandwich Place,Pharmacy,Pizza Place,Fast Food Restaurant,Distribution Center,Dessert Shop
90,Scarborough,4,Fast Food Restaurant,Electronics Store,Coffee Shop,Bank,Pharmacy,Pizza Place,Breakfast Spot,Supermarket,Chinese Restaurant,Sandwich Place


## 7.  Conclusion


From the above analysis, we see that the neighborhoods of Toronto can be clustered into five groups according to the similarity of the composition of the types of venues within those neighborhoods.